# load library

In [ ]:
!pip install timm

In [ ]:
import timm
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Parameter
import torch.optim
import torch.optim as optim
import torch.nn as nn
import torch.nn.parallel

import torchvision.models as models
import torchvision.transforms as transforms

import torch.utils.data as Data
import torch.backends.cudnn as cudnn

#numpy
import numpy as np

#util
import pickle
from PIL import Image
from tqdm.auto import tqdm
from pprint import pprint
import random

# hyperparameter

In [ ]:
IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)

image_size = 224

use_gpu = torch.cuda.is_available()

image_size = 224

batch_size = 64

num_classes = 4

workers = 4

max_epochs = 1

# utility

In [ ]:
class Warp(object):
    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = int(size)
        self.interpolation = interpolation

    def __call__(self, img):
        return img.resize((self.size, self.size), self.interpolation)

    def __str__(self):
        return self.__class__.__name__ + ' (size={size}, interpolation={interpolation})'.format(size=self.size,
                                                                                                interpolation=self.interpolation)
class MultiScaleCrop(object):

    def __init__(self, input_size, scales=None, max_distort=1, fix_crop=True, more_fix_crop=True):
        self.scales = scales if scales is not None else [1, 875, .75, .66]
        self.max_distort = max_distort
        self.fix_crop = fix_crop
        self.more_fix_crop = more_fix_crop
        self.input_size = input_size if not isinstance(input_size, int) else [input_size, input_size]
        self.interpolation = Image.BILINEAR

    def __call__(self, img):
        im_size = img.size
        crop_w, crop_h, offset_w, offset_h = self._sample_crop_size(im_size)
        crop_img_group = img.crop((offset_w, offset_h, offset_w + crop_w, offset_h + crop_h))
        ret_img_group = crop_img_group.resize((self.input_size[0], self.input_size[1]), self.interpolation)
        return ret_img_group

    def _sample_crop_size(self, im_size):
        image_w, image_h = im_size[0], im_size[1]

        # find a crop size
        base_size = min(image_w, image_h)
        crop_sizes = [int(base_size * x) for x in self.scales]
        crop_h = [self.input_size[1] if abs(x - self.input_size[1]) < 3 else x for x in crop_sizes]
        crop_w = [self.input_size[0] if abs(x - self.input_size[0]) < 3 else x for x in crop_sizes]

        pairs = []
        for i, h in enumerate(crop_h):
            for j, w in enumerate(crop_w):
                if abs(i - j) <= self.max_distort:
                    pairs.append((w, h))

        crop_pair = random.choice(pairs)
        if not self.fix_crop:
            w_offset = random.randint(0, image_w - crop_pair[0])
            h_offset = random.randint(0, image_h - crop_pair[1])
        else:
            w_offset, h_offset = self._sample_fix_offset(image_w, image_h, crop_pair[0], crop_pair[1])

        return crop_pair[0], crop_pair[1], w_offset, h_offset

    def _sample_fix_offset(self, image_w, image_h, crop_w, crop_h):
        offsets = self.fill_fix_offset(self.more_fix_crop, image_w, image_h, crop_w, crop_h)
        return random.choice(offsets)

    @staticmethod
    def fill_fix_offset(more_fix_crop, image_w, image_h, crop_w, crop_h):
        w_step = (image_w - crop_w) // 4
        h_step = (image_h - crop_h) // 4

        ret = list()
        ret.append((0, 0))  # upper left
        ret.append((4 * w_step, 0))  # upper right
        ret.append((0, 4 * h_step))  # lower left
        ret.append((4 * w_step, 4 * h_step))  # lower right
        ret.append((2 * w_step, 2 * h_step))  # center

        if more_fix_crop:
            ret.append((0, 2 * h_step))  # center left
            ret.append((4 * w_step, 2 * h_step))  # center right
            ret.append((2 * w_step, 4 * h_step))  # lower center
            ret.append((2 * w_step, 0 * h_step))  # upper center

            ret.append((1 * w_step, 1 * h_step))  # upper left quarter
            ret.append((3 * w_step, 1 * h_step))  # upper right quarter
            ret.append((1 * w_step, 3 * h_step))  # lower left quarter
            ret.append((3 * w_step, 3 * h_step))  # lower righ quarter

        return ret


    def __str__(self):
        return self.__class__.__name__

# load data

In [ ]:
with open("/content/drive/MyDrive/pickle/train_color_data_annotation_v2.pkl","rb") as fr:
    data_list = pickle.load(fr)

In [ ]:
encoding_label_category = {
    "상의":0,
    "하의":1,
    "아우터":2,
    "원피스":3
}

In [ ]:
class CategoryDataset(Data.Dataset):
    def __init__(self, data_list, transform, num_classes=4):
        self.img_list = data_list
        self.transform = transform
        self.num_classes = num_classes

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, index):
        item = self.img_list[index]
        return self.get(item)

    def get(self, item):
        filename = item['file_name']
        labels = encoding_label_category[item['labels']]

        img = Image.open(filename).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)

        target = torch.tensor([labels])

        return img, target

In [ ]:
normalize = transforms.Normalize(mean =IMAGENET_DEFAULT_MEAN, std = IMAGENET_DEFAULT_STD)

train_transform = transforms.Compose([
    MultiScaleCrop(image_size, scales = (1.0,0.875,0.75, 0.66,0.5), max_distort = 2),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

val_transform = transforms.Compose([
    Warp(image_size),
    transforms.ToTensor(),
    normalize,
])

In [ ]:
data_list[0]

{'하의': '/content/drive/MyDrive/custom_fashion_data/category1/1.jpg',
 '상의': '/content/drive/MyDrive/custom_fashion_data/category1/2.jpg'}

In [ ]:
new_data_list = []

for data in tqdm(data_list):

    for key in data:
        
        train_data = {}

        train_data['file_name'] = data[key]
        train_data['labels'] = key

        new_data_list.append(train_data)
    
print(len(new_data_list))

  0%|          | 0/11843 [00:00<?, ?it/s]

17996


In [ ]:
with open("/content/drive/MyDrive/pickle/new_train_category_preprocess.pkl","wb") as fw:
    pickle.dump(new_data_list, fw)

In [ ]:
with open("/content/drive/MyDrive/pickle/new_train_category_preprocess.pkl","rb") as fr:
    new_data_list = pickle.load(fr)

In [ ]:
#create dataset

train_dataset = CategoryDataset(new_data_list,train_transform, num_classes = 4)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size, shuffle=True,
                                            num_workers=workers)

In [ ]:
train_dataset[0]

(tensor([[[-1.5014, -1.5014, -1.5185,  ...,  1.4783,  1.3413,  1.1187],
          [-1.5014, -1.5014, -1.5185,  ...,  1.4612,  1.3413,  1.1187],
          [-1.5014, -1.5014, -1.5185,  ...,  1.4440,  1.3413,  1.1187],
          ...,
          [ 1.2557,  1.2385,  1.2214,  ...,  1.5125,  1.5125,  1.5125],
          [ 1.2728,  1.2557,  1.2385,  ...,  1.5125,  1.5125,  1.5125],
          [ 1.2728,  1.2557,  1.2385,  ...,  1.5125,  1.5125,  1.5125]],
 
         [[-1.4055, -1.4055, -1.4230,  ...,  1.6408,  1.5007,  1.2731],
          [-1.4055, -1.4055, -1.4230,  ...,  1.6232,  1.5007,  1.2731],
          [-1.4055, -1.4055, -1.4230,  ...,  1.6057,  1.5007,  1.2731],
          ...,
          [ 1.4132,  1.3957,  1.3782,  ...,  1.6758,  1.6758,  1.6758],
          [ 1.4307,  1.4132,  1.3957,  ...,  1.6758,  1.6758,  1.6758],
          [ 1.4307,  1.4132,  1.3957,  ...,  1.6758,  1.6758,  1.6758]],
 
         [[-1.1421, -1.1421, -1.1596,  ...,  1.8557,  1.7163,  1.4897],
          [-1.1421, -1.1421,

# define model

In [ ]:
model_names = timm.list_models(pretrained=True)
pprint(model_names)

In [ ]:
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=4)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth


# loss & optimizer

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters() , lr=0.0005)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

# model training

In [ ]:
save_path = '/content/drive/MyDrive/categorymodel_save/category_best_model1.pth'

In [ ]:
train_dataset[0]

In [ ]:
input,target = train_dataset[0]

In [ ]:
input = input.unsqueeze(0)
target = target.unsqueeze(0)

In [ ]:
output = model(input)

In [ ]:
output

tensor([[ 0.3516,  0.1830,  1.6507, -0.0834]], grad_fn=<AddmmBackward0>)

In [ ]:
sum(torch.argmax(output, dim=-1) == target).item()

0

In [ ]:
pred = output.cpu().detach().numpy()
real = target.cpu().detach().numpy()

In [ ]:
pred

array([[ 0.3516418 ,  0.18295833,  1.6506753 , -0.0834136 ]],
      dtype=float32)

In [ ]:
real

array([[1]])

#load best model

In [ ]:
resume = "/content/drive/MyDrive/categorymodel_save/category_best_model1.pth"

checkpoint = torch.load(resume, map_location=torch.device('cpu'))

model.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
#acclerating gpu
torch.backends.cudnn.benchmark = True

model = model.cuda()

# switch to train mode
model.train()

max_epochs = 20

for epoch in range(max_epochs):
    
    epoch_loss = 0.0
    epoch_acc = 0

    print(f"epoch: {epoch}")
    print("-------------------------------------------------------------")

    for i,(input,target) in enumerate(tqdm(train_loader),start = 1):
        
        input = input.cuda()
        target = target.cuda()

        output = model(input)

        loss = criterion(output,target.squeeze(dim=-1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        epoch_acc += sum(torch.argmax(output, dim=-1) == target.squeeze(dim=-1)).item()

        if i % 20 == 0:
            
            print(f"loss : {epoch_loss/i:.4f}")
            print(f"accuracy : {epoch_acc/(i*batch_size):.4f}")

            torch.save(model.state_dict(), save_path)

            print("-------------------------------------------------------------")
            print()

    epoch_loss = epoch_loss/len(train_loader)
    epoch_acc = epoch_acc/(len(train_loader)*batch_size)

    print(f"epoch loss : {epoch_loss:.4f}")
    print(f"accuracy : {epoch_acc:.4f}")

    torch.save(model.state_dict(), save_path)

epoch: 0
-------------------------------------------------------------


  0%|          | 0/282 [00:00<?, ?it/s]

loss : 0.3728
accuracy : 0.8547
-------------------------------------------------------------

loss : 0.3802
accuracy : 0.8516
-------------------------------------------------------------

loss : 0.3782
accuracy : 0.8534
-------------------------------------------------------------

loss : 0.3822
accuracy : 0.8512
-------------------------------------------------------------

loss : 0.3825
accuracy : 0.8495
-------------------------------------------------------------

loss : 0.3808
accuracy : 0.8500
-------------------------------------------------------------

loss : 0.3798
accuracy : 0.8502
-------------------------------------------------------------

loss : 0.3853
accuracy : 0.8475
-------------------------------------------------------------

loss : 0.3890
accuracy : 0.8467
-------------------------------------------------------------

loss : 0.3884
accuracy : 0.8466
-------------------------------------------------------------

loss : 0.3873
accuracy : 0.8474
------------------

  0%|          | 0/282 [00:00<?, ?it/s]

loss : 0.3445
accuracy : 0.8688
-------------------------------------------------------------

loss : 0.3580
accuracy : 0.8645
-------------------------------------------------------------

loss : 0.3567
accuracy : 0.8651
-------------------------------------------------------------

loss : 0.3568
accuracy : 0.8627
-------------------------------------------------------------

loss : 0.3637
accuracy : 0.8588
-------------------------------------------------------------

loss : 0.3672
accuracy : 0.8589
-------------------------------------------------------------

loss : 0.3683
accuracy : 0.8567
-------------------------------------------------------------

loss : 0.3695
accuracy : 0.8562
-------------------------------------------------------------

loss : 0.3717
accuracy : 0.8557
-------------------------------------------------------------

loss : 0.3687
accuracy : 0.8562
-------------------------------------------------------------

loss : 0.3685
accuracy : 0.8553
------------------

...

loss : 0.3038
accuracy : 0.8789
-------------------------------------------------------------

loss : 0.3112
accuracy : 0.8777
-------------------------------------------------------------

loss : 0.3035
accuracy : 0.8831
-------------------------------------------------------------

loss : 0.3007
accuracy : 0.8832
-------------------------------------------------------------

loss : 0.2998
accuracy : 0.8828
-------------------------------------------------------------

loss : 0.2936
accuracy : 0.8840
-------------------------------------------------------------

loss : 0.2920
accuracy : 0.8850
-------------------------------------------------------------

loss : 0.2955
accuracy : 0.8834
-------------------------------------------------------------

loss : 0.2934
accuracy : 0.8842
-------------------------------------------------------------

loss : 0.2952
accuracy : 0.8829
-------------------------------------------------------------

loss : 0.2946
accuracy : 0.8837
------------------

  0%|          | 0/282 [00:00<?, ?it/s]

loss : 0.2804
accuracy : 0.8891
-------------------------------------------------------------

loss : 0.2947
accuracy : 0.8840
-------------------------------------------------------------

loss : 0.2924
accuracy : 0.8818
-------------------------------------------------------------

loss : 0.2896
accuracy : 0.8818
-------------------------------------------------------------

loss : 0.2974
accuracy : 0.8798
-------------------------------------------------------------

loss : 0.3023
accuracy : 0.8781
-------------------------------------------------------------

loss : 0.3022
accuracy : 0.8776
-------------------------------------------------------------

loss : 0.3027
accuracy : 0.8780
-------------------------------------------------------------

loss : 0.3011
accuracy : 0.8789
-------------------------------------------------------------

loss : 0.3018
accuracy : 0.8784
-------------------------------------------------------------

loss : 0.3037
accuracy : 0.8786
------------------

  0%|          | 0/282 [00:00<?, ?it/s]

loss : 0.2656
accuracy : 0.8977
-------------------------------------------------------------

loss : 0.2777
accuracy : 0.8926
-------------------------------------------------------------

loss : 0.2888
accuracy : 0.8914
-------------------------------------------------------------

loss : 0.2970
accuracy : 0.8869
-------------------------------------------------------------

loss : 0.2954
accuracy : 0.8867
-------------------------------------------------------------

loss : 0.2916
accuracy : 0.8861
-------------------------------------------------------------

loss : 0.2969
accuracy : 0.8846
-------------------------------------------------------------

loss : 0.2978
accuracy : 0.8847
-------------------------------------------------------------

loss : 0.2975
accuracy : 0.8852
-------------------------------------------------------------

loss : 0.2986
accuracy : 0.8852
-------------------------------------------------------------

loss : 0.3000
accuracy : 0.8846
------------------

In [ ]:
img = Image.open("/content/캡처.JPG").convert('RGB')

In [ ]:
img = val_transform(img)

In [ ]:
img = img.unsqueeze(0)

In [ ]:
output = model(img)

In [ ]:
torch.argmax(output,dim=-1)

tensor([1])

In [ ]:
encoding_label_category = {
    "상의":0,
    "하의":1,
    "아우터":2,
    "원피스":3
}

In [ ]:
sum(torch.argmax(output, dim=-1) == target.squeeze(dim=-1)).item()

21

In [ ]:
target.squeeze(dim=-1)

tensor([2, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 2, 3, 0, 0, 1, 3, 3, 1, 0, 0,
        3, 3, 0, 0, 1, 1, 1, 2, 1, 2, 3, 2, 1, 0, 1, 0, 3, 0, 0, 1, 1, 1, 3, 1,
        0, 1, 2, 0, 3, 1, 1, 0, 2, 1, 1, 0, 1, 1, 2, 3], device='cuda:0')

In [ ]:
torch.argmax(output, dim=-1)

tensor([0, 2, 0, 2, 0, 0, 1, 3, 2, 2, 0, 0, 2, 0, 2, 0, 3, 0, 1, 2, 2, 2, 0, 0,
        2, 3, 3, 0, 1, 0, 3, 2, 0, 2, 3, 0, 3, 3, 1, 0, 0, 0, 3, 3, 3, 0, 0, 3,
        3, 0, 3, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 1, 3, 0], device='cuda:0')